In [ ]:
!pip install beautifulsoup4 pandas lxml Faker

In [ ]:
import requests
import pandas as pd
import re

from bs4 import BeautifulSoup, Tag
from faker import Faker

class MALScraper:
    def __init__(self):
        self.session = requests.Session()
        self.session.headers.update({'User-Agent': Faker().user_agent()})

    def save_dataframe(self, dataframe, filepath):
            dataframe.to_csv(filepath, index=False)
            

    def get_seasonal_anime(self, year=2025, season='winter'):
        url = f"https://myanimelist.net/anime/season/{year}/{season}"
        soup = BeautifulSoup(self.session.get(url).content, 'html.parser')
        data = []
        items = soup.find_all('div', class_=re.compile(r'seasonal-anime'))[:50]

        for item in items:
            if not isinstance(item, Tag):
                continue

            anime = {}
            title = item.select_one('h2 a, .title a')
            if title and isinstance(title, Tag):
                anime['title'] = title.text.strip()
            else:
                continue

            score = item.select_one('.score')
            if score and isinstance(score, Tag) and score.text.strip() not in ['N/A', '-']:
                try:
                    anime['score'] = float(score.text.strip())
                except ValueError:
                    anime['score'] = None
            else:
                anime['score'] = None

            members = item.select_one('.member')
            if members and isinstance(members, Tag):
                m = members.text.strip().replace(',', '')
                if 'M' in m:
                    anime['members'] = int(float(m[:-1]) * 1000000)
                elif 'K' in m:
                    anime['members'] = int(float(m[:-1]) * 1000)
                elif m.isdigit():
                    anime['members'] = int(m)
                else:
                    anime['members'] = None
            else:
                anime['members'] = None

            eps = re.search(r'(\d+)\s*eps?', item.text, re.I)
            anime['episodes'] = int(eps.group(1)) if eps else None

            data.append(anime)

        dataframe = pd.DataFrame(data)
        dataframe.index = pd.RangeIndex(1, len(dataframe) + 1)

        self.save_dataframe(dataframe, f'anime_{season}.csv')

        return dataframe.drop_duplicates(subset='title')


scraper = MALScraper()
dataframe = scraper.get_seasonal_anime(2023, 'fall')

print(f"Found {len(dataframe)} anime")
dataframe
